In [1]:
# Run once to install the Gensim framework for topic modeling
#!pip install gensim --user


In [9]:
import pandas as pd
import gzip
true = True

false = False
def parse(path):
    g=gzip.open(path, 'r')
    for l in g:
        yield eval(l)

def getDataFrame(path):
    i=0
    df ={}
    data = parse(path)
    for d in data:
        df[i] =d
        i += 1
        if i > 100000:
            break
    return pd.DataFrame.from_dict(df, orient ='index')

df = getDataFrame("C:/Users/Kedar/Desktop/AML/Amazon/Home_and_Kitchen.json.gz")

print(df.columns.values)

['overall' 'vote' 'verified' 'reviewTime' 'reviewerID' 'asin'
 'reviewerName' 'reviewText' 'summary' 'unixReviewTime' 'style' 'image']


In [10]:

corpus = df['reviewText'].values
df.head()

,overall,vote,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,style,image
0,5.0,2,True,"08 31, 2010",A3NSN9WOX8470M,0006564224,mmm,"I don't use these for their original use, and ...",Many uses...,1283212800,NaN,NaN
1,5.0,2,True,"04 2, 2010",A2AMX0AJ2BUDNV,0006564224,John R. Welch,"Seems a bit expensive for a plastic bottle, bu...",Dispenser bottle,1270166400,NaN,NaN
2,5.0,NaN,True,"11 5, 2015",A8LUWTIPU9CZB,0560467893,Linda Fahner,"Great product, love it!!",Five Stars,1446681600,NaN,NaN
3,4.0,4,True,"10 29, 2015",AABKIIHAL0L66,0560467893,TheBlueChain,This is a sturdy floating corner shelf! We mo...,"Sturdy Shelf, Poor Installation Instructions",1446076800,NaN,NaN
4,3.0,NaN,True,"09 9, 2015",A3DA0KIQ5OBK5C,0560467893,angelaarden,I purchased 4 of these shelves. they look grea...,Look great - one bad one...,1441756800,NaN,NaN


In [13]:
df.notnull().mean().sort_values()

image             0.008090
vote              0.178718
style             0.741553
summary           0.999870
reviewerName      0.999940
overall           1.000000
verified          1.000000
reviewTime        1.000000
reviewerID        1.000000
asin              1.000000
reviewText        1.000000
unixReviewTime    1.000000
dtype: float64

In [12]:
df.reviewText.fillna("",inplace = True)

In [14]:
import nltk
from nltk.corpus import wordnet
# The WordNet Lemmatizer
lemmatizer = nltk.stem.WordNetLemmatizer()

# Acceptable parts of speech
pos_map = {'N': wordnet.NOUN, 'J': wordnet.ADJ, 'V': wordnet.VERB, 'R': wordnet.ADV }
# Function to reduce corpus to word lemmas, limited to nouns, verbs, etc. 
def clean_corpus(corpus):
    cleaned = []
    for text in corpus:
        words = []
        for sent in nltk.sent_tokenize(text):
            for (word, pos) in nltk.pos_tag(nltk.word_tokenize(sent)):
                if not pos[0] in pos_map:
                    continue
                word = lemmatizer.lemmatize(word.lower(), pos_map[pos[0]])
                words.append(word.lower())
        cleaned.append(words)
    return cleaned
clean_texts = clean_corpus(corpus)

In [15]:
from collections import Counter

# Print the 30 most frequent words, and add to stopword list
counter = Counter([w for t in clean_texts for w in t])
wordcounts = counter.most_common(n=30)
stopwords = []
for word, count in wordcounts:
    stopwords.append(word)
    print('%s%i' % (word.ljust(15), count))

be             181383
have           76578
use            48487
do             46157
not            34043
n't            32868
make           30402
great          28618
work           25027
very           24820
good           24270
get            23404
's             21492
so             21026
well           20181
buy            19578
just           19520
love           18929
time           18300
easy           18111
product        15623
up             15141
year           14566
more           12382
out            11998
clean          11693
only           11271
much           10293
little         10188
go             10127


In [16]:
# Remove stopwords from each lemmatized text block
for i in range(len(clean_texts)):
    j = 0
    while j < len(clean_texts[i]):
        if clean_texts[i][j] in stopwords:
            del clean_texts[i][j]
        else:
            j += 1

In [17]:
# Show example text block after data preparation
print(clean_texts[0])

['original', 'purchase', 'need', 'replacement', 'last', 'never', 'even', 'think', 'internet', 'find', 'new', 'one', 'pleasant', 'surprise', 'find', 'immediately', 'actually', 'pay', 'le']


In [18]:
from gensim.corpora.dictionary import Dictionary

# Create a corpus from a list of texts
dictionary = Dictionary(clean_texts)
corpus = [dictionary.doc2bow(text) for text in clean_texts]

In [19]:
from gensim.models.ldamulticore import LdaMulticore

# Pretrain the model using the corpus and number of topics
num_topics = 20
lda = LdaMulticore(corpus, id2word=dictionary, num_topics=num_topics, passes=10)

In [30]:
from gensim.models import CoherenceModel
# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda, texts=clean_texts, dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Coherence Score:  0.45741329860959173


In [21]:
# Print the top 20 words in each learned topic
for i in range(num_topics):
    print('Topic %i: %s\n' % (i, lda.print_topic(i, topn=20)))

Topic 0: 0.065*"*" + 0.036*"loaf" + 0.021*"cupcake" + 0.017*"say" + 0.013*"machine" + 0.009*"song" + 0.008*"scale" + 0.007*"show" + 0.007*"eminem" + 0.007*"album" + 0.005*"people" + 0.005*"hear" + 0.005*"cd" + 0.004*"really" + 0.004*"know" + 0.004*"bread" + 0.004*"track" + 0.004*"come" + 0.004*"think" + 0.004*"first"

Topic 1: 0.021*"cake" + 0.012*"small" + 0.008*"too" + 0.008*"perfect" + 0.008*"need" + 0.008*"set" + 0.008*"kitchen" + 0.008*"thing" + 0.007*"also" + 0.007*"timer" + 0.007*"job" + 0.007*"size" + 0.007*"enough" + 0.006*"cookie" + 0.006*"room" + 0.006*"keep" + 0.006*"fan" + 0.006*"come" + 0.005*"really" + 0.005*"large"

Topic 2: 0.035*"espresso" + 0.027*"canner" + 0.026*"nice" + 0.022*"recommend" + 0.021*"look" + 0.017*"quality" + 0.015*"price" + 0.013*"sturdy" + 0.013*"size" + 0.012*"highly" + 0.012*"sharpen" + 0.011*"fit" + 0.011*"exactly" + 0.010*"need" + 0.009*"ball" + 0.008*"small" + 0.008*"steel" + 0.008*"too" + 0.007*"want" + 0.007*"money"

Topic 3: 0.025*"it" + 0.02

In [23]:
from gensim.test.utils import datapath
from gensim import models 
# Save the pretrained model to the datapath
dp = datapath("lda.model")
lda.save(dp)

# Load the pretrained model from the datapath
lda = models.ldamodel.LdaModel.load(dp)

## Exporting model using Pickle

In [24]:
import pickle
# save the model to disk
with open('C:/Users/Kedar/Desktop/AML/Amazon/ama.model','wb') as ama:
    pickle.dump(lda,ama)